<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/get_wordle_titles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **get some titles by wordle word**
<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

This is what can feed a twitter, "tweet a random title" feed

Data sources / Documentation:

* https://ilsweb.cincinnatilibrary.org/collection-analysis/

In [5]:
import requests
import json
from uuid import uuid4
from time import sleep
from IPython.display import Image
import imghdr
from IPython.display import Image

In [6]:
chpl_collection_url = 'https://ilsweb.cincinnatilibrary.org/collection-analysis/current_collection'

In [9]:
wordle_word = 'whack'

In [10]:
sql = """\
with match_data as (
  select
    rowid
  from
    bib_fts
  where
    bib_fts.best_title match escape_fts(:search)
  order by
    bm25(bib_fts) 
    -- random()
)
select
  bib.*,
  sum(item.checkout_total) as sum_item_checkouts,
  min(item.item_format)
from
  match_data
  join bib on bib.rowid = match_data.rowid
  join item on item.bib_record_num = bib.bib_record_num
group by
  1,
  2
having
  min(item.item_format) in (
    'Book',
    'Juvenile Book',
    'Reference Book',
    'DVD/Videocassette',
    'Magazine',
    'Music on CD',
    'Teen Book',
    'Large Print Book',
    'Downloadable Book'
  )
order by
  -- sum(item.checkout_total) DESC
  match_data.rowid
limit
  5
"""

# Note: the random search may not return a value, so loop up to 100 times
# Note: we're sending a guid so that we can make sure the query is not being cached
count = 100
while count:
  count -= 1
  # print('.', end='')

  try:
      r = requests.get(
          chpl_collection_url + '.json',
          params={
              'sql': sql,
              '_shape': 'array',
              # 'guid': str(uuid4()),
              'search': wordle_word
          }
      )
      if len(r.json()) > 0:
        break
      
      df = pd.read_json(r.text)
      
  except:
      pass

  sleep(1)

for title in r.json():
  print(title['best_title'], '—', title['best_author'], '\n', 'https://cincinnatilibrary.bibliocommons.com/v2/record/S170C', title['bib_record_num'], sep='')

# print(
#     "\n",
#     json.dumps(r.json(), indent=2),
#     sep=""
# )

11 tracks of whack [sound recording]—Becker, Walter, 1950-
https://cincinnatilibrary.bibliocommons.com/v2/record/S170C1540481
Whack your porcupine, and other drawings—Kliban, B.
https://cincinnatilibrary.bibliocommons.com/v2/record/S170C1538818
A whack on the side of the head : how to unlock your mind for innovation—Von Oech, Roger.
https://cincinnatilibrary.bibliocommons.com/v2/record/S170C2024094
Knick knack paddy whack—Engel, Christiane (Illustrator)
https://cincinnatilibrary.bibliocommons.com/v2/record/S170C2590022
Knick knack paddy whack : a novel—O'Hanlon, Ardal.
https://cincinnatilibrary.bibliocommons.com/v2/record/S170C1996993
